In [ ]:
import requests
import json
from pathlib import Path

# Transform points to nearest Street View panorama ID
def annotate_streetview(sampled_df):
    sv_lat = []
    sv_lng = []
    sv_pano = []
    sv_status = []
    for index, point in sampled_df.iterrows():
        # Call the Street View API to get the nearest panorama
        result = requests.get(
            "https://maps.googleapis.com/maps/api/streetview/metadata?location={},{}&key={}".format(
                    point['orig_lat'],
                    point['orig_lng'],
                    GMAPS_API_KEY
                )
            ).json()
        sv_status.append(result["status"])
        sv_lat.append(result.get("location", {}).get("lat"))
        sv_lng.append(result.get("location", {}).get("lng"))
        sv_pano.append(result.get("pano_id"))

    df = sampled_df.copy()
    df['lat'] = sv_lat
    df['lng'] = sv_lng
    df['pano_id'] = sv_pano
    df['status'] = sv_status
    return df

# Download panorama images from street view
def streetview_write_point(df_row, output_dir, index):
    output_dir = Path(output_dir)

    which_split = df_row["split"]

    for heading in [0, 45, 90, 135, 180, 225, 270, 315]:
        metadata = df_row.to_dict()
        metadata["heading"] = heading
        metadata["fov"] = 45
        metadata["pitch"] = 0

        url = "https://maps.googleapis.com/maps/api/streetview?size=640x640&pano={pano_id}&fov={fov}&heading={heading}".format(**metadata)
        url += "&key={}".format(GMAPS_API_KEY)
        # TODO: vary the heading, pitch, and FOV

        # Store image & metadata in webdataset format
        # TODO: use the official library?
        jpeg_filename = output_dir / which_split / f"loc_{index}_{heading}.jpg"
        json_filename = output_dir / which_split / f"loc_{index}_{heading}.json"
        if not os.path.exists(jpeg_filename):
            print("Downloading", jpeg_filename)
            r = requests.get(url, allow_redirects=True)
            with open(jpeg_filename, 'wb') as f:
                f.write(r.content)
        else:
            print("Skipping", jpeg_filename)

        with open(json_filename, 'w') as f:
            json.dump(metadata, f)